In [2]:
###Import required packages
from models import loss_biological, fa_layerednegativemetabolicloop, fa_negativegeneloop, fa_negativemetabolicloop, fa_openloop, fa_openloopintermediate
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import pandas as pd

In [3]:
###Specify if data is to be saved to CSV
save_data = True

In [4]:
###Search space definition
space = hp.choice('architecture',
    [('Open Loop', hp.uniform('r_lac_ol', 10E-11, 10E-8)),
        ('Negative Gene Loop', [hp.uniform('r_tl_ngl', 10E-11, 10E-8),hp.uniform('r_tl_tetR_ngl', 10E-11, 10E-8)]),
        ('Negative Metabolic Loop', [hp.uniform('r_fl_prime_nml', 10E-11, 10E-8), hp.uniform('ki_nml', 0., 0.12)]),
        ('Layered Negative Metabolic Loop', [hp.uniform('r_tl_lnml', 10E-11, 10E-8), hp.uniform('r_ar2_lnml', 10E-11, 10E-8),])
        ])

###Objective function
def run_opt(max_iters):
    losses = []
    params = []
    circuits = []

    def objective(args):
        architecture, param_values = args
        #Integration conditions
        t = np.linspace(0, 5E4, 200)

        #Select architecture and generate function
        if architecture == 'Open Loop':
            ode_function = fa_openloop
            y0 = np.array([0., 0., 0., 0.])
        elif architecture == 'Negative Gene Loop':
            ode_function = fa_negativegeneloop
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Metabolic Loop':
            ode_function = fa_negativemetabolicloop
            y0 = np.array([0., 0., 0., 0.])
        else:
            ode_function = fa_layerednegativemetabolicloop
            y0 = np.array([0., 0., 0., 0., 0.])
        
        extra_options = {'old_api': False, 'user_data': param_values}
        ode_solver = ode('cvode', ode_function, **extra_options)
        solution = ode_solver.solve(t, y0)

        j1 = solution.values.y[-1][-1]
        v_prod = solution.values.y[-1][-2]
        j1, j2, loss = loss_biological(j1, 1/v_prod, alpha1=10E3, alpha2=10E-2)

        losses.append(loss)
        params.append(param_values)
        circuits.append(architecture)
        return loss

    #Run hyperopt call
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_iters)
    
    #Create trajectory data frame
    r_lac_ols, r_tl_ngls, r_tl_tetR_ngls, r_fl_prime_nmls, ki_nmls, r_tl_lnmls, r_ar2_lnmls = [[], [], [], [], [], [], []]
    for i in range(max_iters):
        r_lac_ol, r_tl_ngl, r_tl_tetR_ngl, r_fl_prime_nml, ki_nml, r_tl_lnml, r_ar2_lnml = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        arch = circuits[i]
        if arch == 'Open Loop':
            r_lac_ol = params[i]
        elif arch == 'Negative Gene Loop':
            r_tl_ngl = params[i][0]
            r_tl_tetR_ngl = params[i][1]
        elif arch == 'Negative Metabolic Loop':
            r_fl_prime_nml = params[i][0]
            ki_nml = params[i][1]
        else:
            r_tl_lnml = params[i][0]
            r_ar2_lnml = params[i][1]
        
        r_lac_ols.append(r_lac_ol)
        r_tl_ngls.append(r_tl_ngl)
        r_tl_tetR_ngls.append(r_tl_tetR_ngl)
        r_fl_prime_nmls.append(r_fl_prime_nml)
        ki_nmls.append(ki_nml)
        r_tl_lnmls.append(r_tl_lnml)
        r_ar2_lnmls.append(r_ar2_lnml)
        
    landscape = pd.DataFrame({'circuit':circuits, 'loss': losses, 'r_lac_ol':r_lac_ols,
            'r_tl_ngl':r_tl_ngls, 'r_tl_tetR_ngl':r_tl_tetR_ngls, 'r_fl_prime_nml':r_fl_prime_nmls,
            'ki_nml':ki_nmls, 'r_tl_lnml':r_tl_lnmls, 'r_ar2_lnml':r_ar2_lnmls})    

    best_loss = 1E5
    best_circuit = 'Initial'
    best_losses = []
    best_losses_circuits = []
    for i in range(len(landscape)):
        if landscape.loss[i] < best_loss:
            best_loss = landscape.loss[i]
            best_circuit = landscape.circuit[i]
        best_losses.append(best_loss)
        best_losses_circuits.append(best_circuit)
    landscape['best_losses'] = best_losses
    landscape['best_loss_circuit'] = best_losses_circuits
    
    return landscape, best

In [5]:
###Run sample optimization
max_iters = 1000
landscape, best = run_opt(max_iters)
landscape = landscape.reset_index()
if save_data: landscape.to_csv('../data/fatty_acid_sample_run_production_burden.csv')

100%|██████████| 1000/1000 [00:12<00:00, 77.27trial/s, best loss: 0.1442107202956086] 


In [48]:
###Search space definition
global alpha
space = hp.choice('architecture',
    [('Open Loop', hp.uniform('r_lac_ol', 10E-11, 10E-8)),
        ('Negative Gene Loop', [hp.uniform('r_tl_ngl', 10E-11, 10E-8),hp.uniform('r_tl_tetR_ngl', 10E-11, 10E-8)]),
        ('Negative Metabolic Loop', [hp.uniform('r_fl_prime_nml', 10E-11, 10E-8), hp.uniform('ki_nml', 0., 0.12)]),
        ('Layered Negative Metabolic Loop', [hp.uniform('r_tl_lnml', 10E-11, 10E-8), hp.uniform('r_ar2_lnml', 10E-11, 10E-8),])
        ])

###Objective function
def run_opt(max_iters):
    losses = []
    params = []
    circuits = []
    overshoots = []
    rise_times = []

    def objective(args):
        architecture, param_values = args
        #Integration conditions
        t = np.linspace(0, 5E4, 200)

        #Select architecture and generate function
        if architecture == 'Open Loop':
            ode_function = fa_openloop
            y0 = np.array([0., 0., 0., 0.])
        elif architecture == 'Negative Gene Loop':
            ode_function = fa_negativegeneloop
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Metabolic Loop':
            ode_function = fa_negativemetabolicloop
            y0 = np.array([0., 0., 0., 0.])
        else:
            ode_function = fa_layerednegativemetabolicloop
            y0 = np.array([0., 0., 0., 0., 0.])
        
        extra_options = {'old_api': False, 'user_data': param_values}
        ode_solver = ode('cvode', ode_function, **extra_options)
        solution = ode_solver.solve(t, y0)
        
        ffa_traj = solution.values.y[:, 0]
        ffa_ss = solution.values.y[-1][0]
        itemindex = np.where(ffa_traj >= ffa_ss*0.5)[0][0]
        rise_time = 100*(itemindex/100000) #number of samples
        rise_times.append(rise_time)
        ffa_max = np.max(ffa_traj)
        overshoot = 100*(ffa_max - ffa_ss)/ffa_ss
        overshoots.append(overshoot)
        loss = overshoot + alpha*rise_time

        losses.append(loss)
        params.append(param_values)
        circuits.append(architecture)
        return loss

    #Run hyperopt call
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_iters)
    
    #Create trajectory data frame
    r_lac_ols, r_tl_ngls, r_tl_tetR_ngls, r_fl_prime_nmls, ki_nmls, r_tl_lnmls, r_ar2_lnmls = [[], [], [], [], [], [], []]
    for i in range(max_iters):
        r_lac_ol, r_tl_ngl, r_tl_tetR_ngl, r_fl_prime_nml, ki_nml, r_tl_lnml, r_ar2_lnml = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        arch = circuits[i]
        if arch == 'Open Loop':
            r_lac_ol = params[i]
        elif arch == 'Negative Gene Loop':
            r_tl_ngl = params[i][0]
            r_tl_tetR_ngl = params[i][1]
        elif arch == 'Negative Metabolic Loop':
            r_fl_prime_nml = params[i][0]
            ki_nml = params[i][1]
        else:
            r_tl_lnml = params[i][0]
            r_ar2_lnml = params[i][1]
        
        r_lac_ols.append(r_lac_ol)
        r_tl_ngls.append(r_tl_ngl)
        r_tl_tetR_ngls.append(r_tl_tetR_ngl)
        r_fl_prime_nmls.append(r_fl_prime_nml)
        ki_nmls.append(ki_nml)
        r_tl_lnmls.append(r_tl_lnml)
        r_ar2_lnmls.append(r_ar2_lnml)
        
    landscape = pd.DataFrame({'circuit':circuits, 'loss': losses, 'r_lac_ol':r_lac_ols,
            'r_tl_ngl':r_tl_ngls, 'r_tl_tetR_ngl':r_tl_tetR_ngls, 'r_fl_prime_nml':r_fl_prime_nmls,
            'ki_nml':ki_nmls, 'r_tl_lnml':r_tl_lnmls, 'r_ar2_lnml':r_ar2_lnmls, 'rise_time': rise_times, 'overshoot':overshoots, 'alpha': np.ones(len(overshoots))*alpha})    

    best_loss = 1E5
    best_circuit = 'Initial'
    best_losses = []
    best_losses_circuits = []
    for i in range(len(landscape)):
        if landscape.loss[i] < best_loss:
            best_loss = landscape.loss[i]
            best_circuit = landscape.circuit[i]
        best_losses.append(best_loss)
        best_losses_circuits.append(best_circuit)
    landscape['best_losses'] = best_losses
    landscape['best_loss_circuit'] = best_losses_circuits
    
    return landscape, best

In [57]:
###Sweep alpha in a logwise fashion
alphas = np.logspace(-2, 4, num=50)
optimal = pd.DataFrame()
for a in alphas:
    max_iters = 100
    alpha = a
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    l_opt = landscape.loc[landscape.loss == landscape.loss.min()]
    optimal = pd.concat([optimal, l_opt])
optimal.to_csv('../data/fatty_acid_pareto_curve_speed_accuracy.csv')

100%|██████████| 100/100 [00:01<00:00, 62.10trial/s, best loss: 2481.5356418642273]


In [74]:
###More closely sample transition zones with linspace
alphas = np.linspace(3.72, 4.94, 27)

#optimal = pd.DataFrame()
for a in alphas[1:-1]:
    max_iters = 100
    alpha = a
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    l_opt = landscape.loc[landscape.loss == landscape.loss.min()]
    optimal = pd.concat([optimal, l_opt])

alphas = np.linspace(255.95, 339.32, 27)
for a in alphas[1:-1]:
    max_iters = 100
    alpha = a
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    l_opt = landscape.loc[landscape.loss == landscape.loss.min()]
    optimal = pd.concat([optimal, l_opt])
    
alphas = np.linspace(275, 10E4, 100)
for a in alphas[1:-1]:
    max_iters = 100
    alpha = a
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    l_opt = landscape.loc[landscape.loss == landscape.loss.min()]
    optimal = pd.concat([optimal, l_opt])
optimal.to_csv('../data/fatty_acid_pareto_curve_speed_accuracy.csv')

100%|██████████| 100/100 [00:01<00:00, 78.86trial/s, best loss: 387.69032477642946]


In [8]:
def objective(args):
    architecture, param_values = args
    #Integration conditions
    sampling_rate = 10000
    t = np.linspace(0, 5E4, sampling_rate)

    #Select architecture and generate function
    if architecture == 'Open Loop':
        ode_function = fa_openloop
        y0 = np.array([0., 0., 0., 0.])
    elif architecture == 'Negative Gene Loop':
        ode_function = fa_negativegeneloop
        y0 = np.array([0., 0., 0., 0., 0.])
    elif architecture == 'Negative Metabolic Loop':
        ode_function = fa_negativemetabolicloop
        y0 = np.array([0., 0., 0., 0.])
    else:
        ode_function = fa_layerednegativemetabolicloop
        y0 = np.array([0., 0., 0., 0., 0.])
    
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    
    return solution
